<a href="https://colab.research.google.com/github/naokityokoyama/NLP_Embedding_Transformers/blob/main/NLP_Embedding_pre_treinado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### import lib

In [ ]:
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Dense, Flatten, LSTM, Dropout, GRU
from keras.utils import np_utils

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import numpy as np
import pandas as pd
import requests
from tqdm import tqdm

### datasets

In [ ]:
train = 'https://raw.githubusercontent.com/UniversalDependencies/UD_Portuguese-Bosque/master/pt_bosque-ud-train.conllu'
dev = 'https://raw.githubusercontent.com/UniversalDependencies/UD_Portuguese-Bosque/master/pt_bosque-ud-dev.conllu'
test = 'https://raw.githubusercontent.com/UniversalDependencies/UD_Portuguese-Bosque/master/pt_bosque-ud-test.conllu'

### embedding

In [ ]:
cbow = '/content/drive/MyDrive/Dataset/cbow_s50.txt'

In [ ]:
response = requests.get(dev)
url = response.text

In [ ]:
class Coleta:
  def __init__(self, url):
    self.url = url
    
  def extrair(self, tamanho = len(url.split('\n'))):
    data_1 = []
    data_2 = []
    data_3 = []
    for i in tqdm(range(tamanho)):  
      data1 = self.url.split('\n')[i].split('\t')
      try:
        if (data1[0][0])!= '#':
          data_1.append(data1[0])
          data_2.append(data1[2])
          data_3.append(data1[3])
      except:
        pass    
    return data_1, data_2, data_3

  def dataframe(self, tamanho=len(url.split('\n'))):

    lista1 = self.extrair(tamanho=tamanho)[0]
    lista2 = self.extrair(tamanho=tamanho)[1]
    lista3 = self.extrair(tamanho=tamanho)[2]
    df = pd.concat([pd.DataFrame(lista1), pd.DataFrame(lista2), pd.DataFrame(lista3)], axis=1)       
    df.columns = ['id', 'token', 'POS_']
    return df
  

### load dataset

In [ ]:
dados = Coleta(url)
df = dados.dataframe()

100%|██████████| 35443/35443 [03:32<00:00, 166.92it/s]


In [ ]:
X = df['token'].values
y = df['POS_'].values

In [ ]:
le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
y = np_utils.to_categorical(y.tolist())


### tokenizer

In [ ]:
token = Tokenizer()
token.fit_on_texts(X)
vocab_size = len(token.word_index) +1 

In [ ]:
vocab_size 

5194

In [ ]:
X = token.texts_to_sequences(X)
print(X)

[[167], [8], [673], [], [2], [1], [1321], [42], [15], [8], [2163], [147], [4], [1], [1322], [209], [8], [7], [2164], [], [2], [1], [32], [51], [2165], [2166], [2167], [1], [26], [674], [8], [7], [2168], [2], [519], [], [3], [1], [2169], [107], [], [1323], [89], [62], [926], [23], [8], [1324], [], [11], [1], [17], [148], [2170], [2], [1321], [], [70], [1], [2171], [], [2], [1], [675], [422], [], [3], [1], [348], [], [2], [1], [56], [], [1325], [131], [1], [1326], [2172], [], [676], [520], [2], [2173], [2], [108], [2], [188], [6], [2174], [2175], [677], [6], [297], [], [118], [247], [3], [210], [248], [], [5], [1], [1327], [], [2], [1], [249], [1328], [927], [2176], [], [678], [], [247], [], [3], [1], [99], [679], [248], [], [11], [1], [1327], [2], [211], [], [5], [1], [249], [3], [4], [21], [], [4], [680], [], [3], [1], [210], [2177], [], [1], [928], [28], [], [5], [1], [2178], [], [1329], [], [2], [1], [2179], [929], [], [2], [1], [298], [], [423], [189], [], [], [1], [29], [62], [100]

In [ ]:
max_length = 1 #size the sentence 
X = pad_sequences(X, maxlen=max_length, padding='post')

### Split train / test

In [ ]:
X_train, X_test, y_train , y_test = train_test_split(X,y, test_size=0.25)

In [ ]:
X_train.shape, y_train.shape

((22882, 1), (22882, 17))

### Load cbow / embedding

In [ ]:
#load embedding files 
def load_embedding(filename):
    file = open(filename,'r')
    lines = file.readlines()[1:]
    file.close()
    # create a map of words to vectors
    embedding = dict()
    for line in tqdm(lines):
        parts = line.split()
        # key is string word, value is numpy array for vector
        embedding[parts[0]] = np.array(parts[1:]) #asarray dif ??
    return embedding

### get embedding with word

In [ ]:
def get_weight_matrix(embedding, vocab, size_embedding):
    # total vocabulary size plus 0 for unknown words
  vocab_size = len(vocab) + 1
# define weight matrix dimensions with all 0
  weight_matrix = np.zeros((vocab_size, size_embedding))
  #weight_matrix = {}
# create matrix embedding with words (X)
  for word, i in vocab.items():
    embedding_vector = embedding.get(word)
    try:
      if embedding_vector is not None:
        weight_matrix[i] = embedding.get(word)
    except:
      pass    
  return weight_matrix

### apply funcion

In [ ]:
raw_embedding = load_embedding(cbow)
embedding_vectors = get_weight_matrix(raw_embedding, token.word_index, 50)

100%|██████████| 929606/929606 [00:15<00:00, 60573.46it/s]


In [ ]:
raw_embedding.get('fraco')

array(['0.161167', '-0.093236', '0.030962', '0.331042', '-0.672444',
       '0.092897', '0.245852', '0.098617', '0.019999', '-0.300309',
       '0.152758', '0.125256', '-0.504440', '0.423323', '0.092136',
       '-0.340580', '-0.163691', '-0.074252', '0.022429', '0.258025',
       '-0.027156', '-0.197107', '-0.220700', '0.029477', '-0.082509',
       '-0.038749', '-0.569372', '0.104082', '-0.035778', '0.181943',
       '-0.209470', '-0.070859', '0.113234', '0.251036', '0.028525',
       '0.019925', '0.524409', '-0.240422', '0.046819', '0.100377',
       '0.010617', '0.163828', '-0.460975', '-0.181066', '0.017025',
       '0.051055', '0.180974', '0.073083', '-0.298676', '-0.147611'],
      dtype='<U9')

### train

In [ ]:
model = Sequential()
embedding  = Embedding(vocab_size, output_dim=50, input_length=max_length, trainable=False, weights=[embedding_vectors])
model.add(embedding)
model.add(LSTM(units=10,  activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(units=y.shape[1], activation='softmax'))
# compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['acc'])
# summarize the model
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 1, 50)             259700    
                                                                 
 lstm_5 (LSTM)               (None, 10)                2440      
                                                                 
 dropout_6 (Dropout)         (None, 10)                0         
                                                                 
 dense_6 (Dense)             (None, 17)                187       
                                                                 
Total params: 262,327
Trainable params: 2,627
Non-trainable params: 259,700
_________________________________________________________________


In [ ]:
model.fit(X_train, y_train,  batch_size=32, epochs=10, verbose=1)

Epoch 1/10
716/716 [==============================] - 3s 2ms/step - loss: 0.0491 - acc: 0.3130
Epoch 2/10
716/716 [==============================] - 2s 2ms/step - loss: 0.0387 - acc: 0.4730
Epoch 3/10
716/716 [==============================] - 2s 2ms/step - loss: 0.0363 - acc: 0.5073
Epoch 4/10
716/716 [==============================] - 2s 2ms/step - loss: 0.0351 - acc: 0.5204
Epoch 5/10
716/716 [==============================] - 2s 2ms/step - loss: 0.0343 - acc: 0.5336
Epoch 6/10
716/716 [==============================] - 2s 2ms/step - loss: 0.0334 - acc: 0.5534
Epoch 7/10
716/716 [==============================] - 2s 2ms/step - loss: 0.0327 - acc: 0.5638
Epoch 8/10
716/716 [==============================] - 2s 2ms/step - loss: 0.0322 - acc: 0.5673
Epoch 9/10
716/716 [==============================] - 2s 2ms/step - loss: 0.0317 - acc: 0.5760
Epoch 10/10
716/716 [==============================] - 2s 2ms/step - loss: 0.0316 - acc: 0.5820


### evaluate

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)

239/239 [==============================] - 1s 2ms/step - loss: 0.0309 - acc: 0.5943


In [ ]:
accuracy

0.5942580103874207

In [ ]:
#somente os pesos
#0.8245936036109924
#sem embedding treinado
#0.5942580103874207
#treino com transfer learning
#0.8479287028312683